In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
import os
import seaborn as sns
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%reload_ext rpy2.ipython

In [2]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/CSF/Analysis/Code/DQA/')

In [44]:
def clean_scores(col_list, df):
    for col in col_list:
        df.loc[:, col].fillna(0, inplace=True)
        df.loc[:, col].replace(88,0,inplace=True)
        df.loc[:, col].replace('88',0,inplace=True)
        df.loc[:, col].replace(999,0,inplace=True)
        df.loc[:, col].replace('999',0,inplace=True)
        df.loc[:, col].replace('UNDEFINED', 0, inplace=True)
        df.loc[:, col].replace('SKIPPED', 0, inplace=True)
        df.loc[:, col].replace('.', 0, inplace=True)
        df.loc[:, col] = df.loc[:, col].astype('int64')
    return

<h1>
    Import raw data files
</h1>

In [66]:
#Importing all scored files
mp_literacy_scored = pd.read_excel("./2022_11_08_mp_literacy_total_scores.xlsx", index_col=[0])
mp_numeracy_scored = pd.read_excel("./2022_11_08_mp_numeracy_total_scores.xlsx", index_col=[0])
up_literacy_scored = pd.read_excel("./2022_11_08_up_literacy_total_scores.xlsx", index_col=[0])
up_numeracy_scored = pd.read_excel("./2022_11_08_up_numeracy_total_scores.xlsx", index_col=[0])

In [3]:
up_literacy_raw = pd.read_excel("./2022_11_08_up_raw_literacy_cleaned.xlsx", index_col=[0])
up_numeracy_raw = pd.read_excel("./2022_11_08_up_raw_numeracy_cleaned.xlsx", index_col=[0])
mp_literacy_raw = pd.read_excel("./2022_11_08_mp_raw_literacy_cleaned.xlsx", index_col=[0])
mp_numeracy_raw = pd.read_excel("./2022_11_08_mp_raw_numeracy_cleaned.xlsx", index_col=[0])

In [5]:
# all(col in up_literacy_scored.columns for col in mp_literacy_scored.columns)
# all(col in up_numeracy_scored.columns for col in mp_numeracy_scored.columns)
all(col in up_literacy_raw.columns for col in mp_literacy_raw.columns)
all(col in up_numeracy_raw.columns for col in mp_numeracy_raw.columns)

True

In [54]:
#Merging state files
literacy_scored = pd.concat([mp_literacy_scored,up_literacy_scored],join='inner')
numeracy_scored = pd.concat([mp_numeracy_scored,up_numeracy_scored],join='inner')

In [56]:
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')                                                                                                    

In [57]:
#Output for Tableau Workbook
literacy_scored.to_excel("../Results/literacy_scored.xlsx")
numeracy_scored.to_excel("../Results/numeracy_scored.xlsx")

In [ ]:
#renaming variables for ease and use in R
literacy_scored = literacy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'})
numeracy_scored = numeracy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'})

In [ ]:
literacy_scored.head()

In [ ]:
literacy_scored['id'] = literacy_scored.index
numeracy_scored['id'] = numeracy_scored.index

In [ ]:
# %%R -i literacy_scored

# require(dplyr)
# require(Hmisc)
# require(sqldf)
# sqldf("select State, Gender, count(distinct id) as ct from literacy_scored group by State, Gender")

In [ ]:
# Check for gender distribution
# literacy_scored.groupby(['State','Gender'])['id'].count()

In [ ]:
# # check for student responses per school greater than 12
# chk1 = pd.DataFrame(literacy_scored.groupby(['UDISE'])['id'].count())
# chk1['UDISE'] = chk1.index
# chk1 = pd.DataFrame(chk1.groupby(['id'])['UDISE'].count())
# chk2 = pd.DataFrame(numeracy_scored.groupby(['UDISE'])['id'].count())
# chk2['UDISE'] = chk2.index
# chk2 = pd.DataFrame(chk2.groupby(['id'])['UDISE'].count())
# chk1.merge(chk2,how='left',on='id')

In [ ]:
# chk = pd.DataFrame(literacy_scored.groupby(['UDISE','Name','Age','Gender'])['id'].count())
# chk['UDISE'] = chk.index
# chk.groupby(['id'])['UDISE'].count()

In [ ]:
# literacy_scored[literacy_scored['literacy9b_total']>7].groupby(['State','District'])['Name'].count()

In [78]:
#Merging state files for item analysis
literacy_raw = pd.concat([mp_literacy_raw,up_literacy_raw],join='inner')
numeracy_raw = pd.concat([mp_numeracy_raw,up_numeracy_raw],join='inner')

In [96]:
literacy_raw['id']= list(literacy_raw.index)
numeracy_raw['id']= list(numeracy_raw.index)

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_87614/13276588.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  numeracy_raw['id']= list(numeracy_raw.index)


In [141]:
main_var = [
 'id',
 'tabletUserName',
 'assessment_date',
 'school_details.State_label',
 'school_details.District_label',
 'school_details.Block_label',
 'school_details.School_label',
 'school_details.UDISE_cd_label',
 'SI_std_name',
 'student_age',
 'student_gender']

In [142]:
literacy_subtask_var = [col for col in literacy_raw.columns 
                if re.search(r'literacy1_q\d$', col)
                or re.search(r'literacy2_q\d+$', col)
                or re.search(r'literacy3_q+', col)
                or re.search(r'literacy4_ut_grid_\d*$', col)
                or re.search(r'literacy4_tt_grid_\d*$', col)
                or re.search(r'literacy5_ut_grid_\d*$', col)
                or re.search(r'literacy5_tt_grid_\d*$', col)
                or re.search(r'literacy6_tt_grid_\d*$', col)
                or re.search(r'literacy7_tt_grid_\d*$', col)
                or re.search(r'literacy8_ut_grid_\d*$', col)
                or re.search(r'literacy8_ut_q\d*$', col)
                or re.search(r'literacy9a_ut_grid_\d*$', col)
                or re.search(r'literacy9b_ut_grid_\d*$', col)  
               ]

In [143]:
numeracy_subtask_var = [col for col in numeracy_raw.columns
                       if re.search(r'numeracy1_tt_grid_\d*$', col)
                       or re.search(r'numeracy2_ut_grid_\d*$', col)
                       or re.search(r'numeracy2_tt_grid_\d*$', col)
                       or re.search(r'numeracy3\w*', col)
                       or re.search(r'numeracy4_ut_q\d*$', col)
                       or re.search(r'numeracy5_ut_q\d*$', col)
                       or re.search(r'numeracy6_ut_q\d*$', col)
                       or re.search(r'numeracy7_ut_q\d*$', col)
                       or re.search(r'numeracy8_ut_q\d*$', col)
                       or re.search(r'numeracy9a_ut_grid_\d$', col)
                       or re.search(r'numeracy9b_ut_grid_\d$', col)
                       ]

In [144]:
literacy_subtask_long = pd.melt(literacy_raw,id_vars = main_var,value_vars = literacy_subtask_var)

In [145]:
list_clean = ['value']

In [146]:
clean_scores(list_clean, literacy_subtask_long)

In [147]:
literacy_subtask_long[['subtask','item']] = literacy_subtask_long['variable'].str.split('_',1,expand=True)

In [148]:
literacy_subtask_long['flag'] = literacy_subtask_long['item'].str[:2]

In [149]:
literacy_subtask_long['timed_untimed'] = np.where(literacy_subtask_long['flag']=='tt','tt','ut')

In [150]:
literacy_subtask_long.to_csv("../Results/literacy_subtask_scored.csv")

In [151]:
numeracy_subtask_long = pd.melt(numeracy_raw, id_vars = main_var, value_vars = numeracy_subtask_var)

In [152]:
clean_scores(list_clean, numeracy_subtask_long)

In [153]:
numeracy_subtask_long[['subtask','item']] = numeracy_subtask_long['variable'].str.split('_',1,expand=True)

In [154]:
numeracy_subtask_long['flag'] = numeracy_subtask_long['item'].str[:2]

In [155]:
numeracy_subtask_long['timed_untimed'] = np.where(numeracy_subtask_long['flag']=='tt','tt','ut')

In [156]:
numeracy_subtask_long.to_csv("../Results/numeracy_subtask_scored.csv")

In [157]:
lst = literacy_subtask_long['variable'].unique()

In [158]:
dfr = pd.DataFrame(lst)

In [159]:
dfr.to_csv("../Results/literacy_var.csv")